In [ ]:
def PLOT(img, anno, bbox_dis = False):
    fig, ax = plt.subplots()
    cmp = ['#ff0000', '#00ff00', '#0000ff', '#ffff00', '#00ffff', '#ff00ff', '#ff8000', '#80ff00']
    skeleton = [[15,13],[13,11],[16,14],[14,12],[11,12],[5,11],[6,12],[5,6],[5,7],[6,8],[7,9],[8,10],[1,2],[0,1],[0,2],[1,3],[2,4],[3,5],[4,6]]
    ax.imshow(img)
    kypt = anno['keypoints']
    anno_x = []
    anno_y = []
    anno_w = []
    if len(kypt[0]) < 3:
        for k in kypt:
            anno_x.append(k[0])
            anno_y.append(k[1])    
        anno_w = anno['kweights']
    else:
        for k in range(int(len(kypt[0])/3)):
            anno_x.append(kypt[0][k*3])
            anno_y.append(kypt[0][k*3+1])
            anno_w.append(kypt[0][k*3+2])
    ax.scatter(anno_x, anno_y, marker = '.', color = cmp[1])
    for i in skeleton:
        #if anno_x[i[0]] != 0 and anno_x[i[1]] != 0 and anno_y[i[0]] != 0 and anno_y[i[1]] != 0:
        if anno_w[i[0]] != 0 and anno_w[i[1]] != 0:
            x = []
            y = []
            x.append(anno_x[i[0]])
            x.append(anno_x[i[1]])
            y.append(anno_y[i[0]])
            y.append(anno_y[i[1]])
            ax.plot(x, y , linestyle = '-', color = cmp[1])
    if bbox_dis:
        bbox = []
        codes = []
        vertices = []
        codes = [Path.MOVETO] + [Path.LINETO]*3 + [Path.CLOSEPOLY]
        bbox.append(anno['bbox'][0][0])
        bbox.append(anno['bbox'][0][1])
        bbox.append(anno['bbox'][0][0] + anno['bbox'][0][2])
        bbox.append(anno['bbox'][0][1] + anno['bbox'][0][3])
        vertices = [(bbox[0], bbox[1]), (bbox[0], bbox[3]), (bbox[2], bbox[3]), (bbox[2], bbox[1]), (0, 0)]
        path = Path(vertices, codes)
        pathpatch = PathPatch(path, facecolor ='none', edgecolor = cmp[1])
        ax.add_patch(pathpatch)
    plt.show()
    #fig.savefig('./output.jpg')
    return

In [ ]:
def PLOT_HEATMAP(heatmap):
    label = ["nose","left_eye","right_eye","left_ear", 
             "right_ear","left_shoulder","right_shoulder",
             "left_elbow","right_elbow","left_wrist","right_wrist",
             "left_hip","right_hip","left_knee","right_knee","left_ankle","right_ankle"]
    for i in range(17):
        plt.subplot(5, 4, i+1)
        plt.imshow(heatmap[i])
        plt.title(label[i])
    plt.tight_layout()
    plt.show()
    return

In [ ]:
def gaussian_heatmap(anno, shape, sigma=8):
    kypt = anno['keypoints']
    heatmap = []
    for i in range(len(kypt)):
        if anno['weights'][i] != 0: 
            coords = kypt[i]
            x, y = np.meshgrid(np.arange(shape[1]), np.arange(shape[0]))
            dists = np.sqrt((x[:, :, np.newaxis] - coords[0])**2 +
                            (y[:, :, np.newaxis] - coords[1])**2)
            gaussians = np.exp(-dists**2 / (2 * sigma**2))
            tmp = np.sum(gaussians, axis=2)
            tmp /= np.max(tmp)
            heatmap.append(tmp)
        else:
            heatmap.append(np.zeros(shape))
    heatmap = np.array(heatmap)
    return heatmap

In [2]:
from PIL import Image
from torchvision.transforms import ToPILImage
import matplotlib.pyplot as plt

In [3]:
from torch.utils.data import DataLoader
import albumentations as A
from CustomDataset import COCO2017Keypoint
from Models import ResNet
import cv2
transforms = A.Compose([A.RandomBrightnessContrast(p = 0.5), 
                        A.SafeRotate(interpolation = cv2.INTER_LINEAR, border_mode = cv2.BORDER_CONSTANT, value = 0, p = 0.5),
                        A.Resize(height=224,
                                    width=224,
                                    interpolation = cv2.INTER_LINEAR),
                        #A.Normalize()
                       ],
                       keypoint_params = A.KeypointParams(format = 'xy', remove_invisible = False))

In [1]:
import torch

In [4]:
from torchsummary import summary

In [5]:
val_loader = DataLoader(dataset = COCO2017Keypoint(PATH = 'PATH.json', 
                                                   transforms = transforms, 
                                                   mode = 'Val'), 
                        batch_size = 1, 
                        shuffle = False, )
                        #num_workers = 1)

loading annotations into memory...
Done (t=0.34s)
creating index...
index created!


In [6]:
network = ResNet(18)

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # PyTorch v0.4.0
network = ResNet(18).to(device)
summary(network, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
       BatchNorm2d-5           [-1, 64, 56, 56]             128
              ReLU-6           [-1, 64, 56, 56]               0
            Conv2d-7           [-1, 64, 56, 56]          36,864
       BatchNorm2d-8           [-1, 64, 56, 56]             128
              ReLU-9           [-1, 64, 56, 56]               0
           Conv2d-10           [-1, 64, 56, 56]          36,864
      BatchNorm2d-11           [-1, 64, 56, 56]             128
       BasicBlock-12           [-1, 64, 56, 56]               0
           Conv2d-13           [-1, 64, 56, 56]          36,864
      BatchNorm2d-14           [-1, 64,

In [ ]:
count = 10
import numpy as np
from torchvision.transforms.functional import to_pil_image
for idx, data in enumerate(val_loader):
    data[0] = network(data[0])
    print(data[0].size())
    #plt.imshow(to_pil_image(data[0].view(3,224,224)))
    #plt.show()
    count -= 1
    if count == 0:
        break

In [ ]:
tmg = Image.open('/root/autodl-tmp/Baseline/data/coco/images/val2017/000000353180.jpg')
tmg2 = Image.open('/root/autodl-tmp/Baseline/data/coco/images/val2017/000000425226.jpg')

In [ ]:
plt.imshow(tmg2)
plt.show()

In [ ]:
import numpy as np
print(np.array(tmg2).shape)

In [ ]:
m1 = torch.zeros(1,10,10)

In [ ]:
m2 = torch.cat([m1, m1, m1], dim =0)

In [ ]:
print(m1.size())
print(m2.size())